In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### We start by setting up coding environment and importing neccessary libraries

In [ ]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv("/kaggle/input/kaggle-survey-2021/kaggle_survey_2021_responses.csv")
df.tail(5)

### One look at the dataset and we note each column barring column 1 which is 'Time from Start to Finish (seconds)' contains Questions coded with index 1 to i, Questions with multiple selectable option are further coded with suffix Part_j(1 - j), we will need to do a little data engineering for sure to consolidate Quesitons with multiple parts.

**We will start by plotting individual columns and try to see if we can derive meaningful insights from it**

In [ ]:
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q1',
              data = df,
              order = df['Q1'].value_counts().index)
plt.show()

print(df.Q1.value_counts())

#### As we may notice, we are able to plot the age group column Q1, with the Question "What is your Age" also being plotted on the grid with a count 1, we don't mind that, we get to read the question and visualise the values via the barplot of value counts.

**We also display a table of Age group.**

- We note that age group 25-29 is at top with 4931 entries followed by 18-21 with 4901 and 22-24 with 4694 entries, 
- now that was expected meaning age group 18 - 29 consolidated contain about 50 % of all entries. 
- That means the age group 30 - 70 is also adopting usage of these technologies and thats a great insight, 
- furthermore the age group of 30 - 55 contains about 45 % all all entreis.

**Now we will plot Q2 and visualise for insights**

In [ ]:
plt.figure(figsize=(15,5))

ax = sn.countplot(x="Q2", data=df)

print(df.Q2.value_counts())
#Overall female respondent percentage
print(4890/25974*100)

**The gender skew is very clearly visible, out of 25974 entreis 20598 are men and 4890 are women, that's about 79 % male respondents**

**We get down to Q3 , which is about the country of respondents**

In [ ]:
# Finding and plotting respondents from each country
sn.set(style='darkgrid')
plt.figure(figsize=(10,15))
sn.countplot(y = 'Q3',
              data = df,
              order = df['Q3'].value_counts().index)
plt.show()

print(df.Q3.value_counts())

#### As we may notice there are total 25974 entries and 

- The respondents from India are almost 3 times that of the far second USA and 6 times that of Japan it seems from this dataset.

- India tops with a total 7434, followed by USA with 2650, Japan 921 and China 814. 

**Now we will try to plot geoclusters of respondents on the worldmap using folium, we will need to assign lat lon data for each country, for that we will use open source information from github with thanks to @@melanieshi0210**

In [ ]:
#Visualising distribution of respondents across world map, Geo clustering using folium
import folium
from folium.plugins import MarkerCluster
# loading location data which contains each country’s latitude and longitude, courtsey: @melanieshi0210
location=pd.read_csv('https://raw.githubusercontent.com/melanieshi0120/COVID-19_global_time_series_panel_data/master/data/countries_latitude_longitude.csv')
location.rename(columns = {'name':'Q3'}, inplace = True)

# merge dataframes and convert column to numeric
merge=pd.merge(location,df,on='Q3')
merge['Time from Start to Finish (seconds)'] = pd.to_numeric(merge['Time from Start to Finish (seconds)'])

world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(merge)):
        lat = merge.iloc[i]['latitude']
        long = merge.iloc[i]['longitude']
        radius=i
        popup_text = """Q3 : {}<br>
                    'Time from Start to Finish (seconds)' : {}<br>"""
        popup_text = popup_text.format(merge.iloc[i]['Q3'],
                                   merge.iloc[i]['Time from Start to Finish (seconds)']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
    
    
world_map

**Now we will try to find specific data from these geoclusters, uniquely identified by countrynames as given in the dataset**

In [ ]:
df_In = df[df.Q3 == 'India']

df_US = df[df.Q3 == 'United States of America']

df_Jp = df[df.Q3 == 'Japan']

df_Cn = df[df.Q3 == 'China']

#Gender distribution for specific countries(Top 4)

print("Gender distribution India: \n",df_In.Q2.value_counts())

print("\nGender distribution Japan: \n",df_Jp.Q2.value_counts())

print("\nGender distribution US: \n",df_US.Q2.value_counts())

print("\nGender distribution China: \n",df_Cn.Q2.value_counts())

In [ ]:
#Female percentage in data science for top 4 countries

print("female_percentage_In :", round(1656/7434*100,2),"\n")

print("female_percentage_Jp :", round(68/921*100,2),"\n")

print("female_percentage_US :", round(606/2650*100,2),"\n")

print("female_percentage_Cn :", round(106/818*100,2),"\n")


**India and US fairs among the top of the list for presence of women in Data Science, they seem to have better than average number of women in Data Science than rest of the world**

Now we get down to Q4 , which is about educational qualification of respondents

In [ ]:
#We visualise responses of Q4

print(df.Q4.value_counts())
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q4',
              data = df,
              order = df['Q4'].value_counts().index)
plt.show()

#### Very clearly visible that about bachelor's and Masters degree level combined have over 20 k entries.

In [ ]:
#Current profession of respondents

print(df.Q5.value_counts())
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q5',
              data = df,
              order = df['Q5'].value_counts().index)
plt.xticks(size = 12)
plt.show()



**With 6804 entries, about 25 % respondents are students, followed by**

- 3616 Data scientists
- 2449 Software engineers
- 2301 Data Analysts

In [ ]:
#Visualising Q6 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q6',
              data = df,
              order = df['Q6'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q6.value_counts())

**

**Visualising Q7 which has multiple parts to it is slightly tricky, we need to consolidate each of these questions with multiple parts to visualise**

In [ ]:
# We make lists containing all columns containing these questions with multiple parts

df2 = pd.read_csv("/kaggle/input/kaggle-survey-2021/kaggle_survey_2021_responses.csv",header=None)

Q7 = list(df2.loc[0][7:20])

Q9 = list(df2.loc[0][21:34])

Q10 = list(df2.loc[0][34:51])

Q12 = list(df2.loc[0][52:58])

Q14 = list(df2.loc[0][59:71])



Q16 = list(df2.loc[0][72:90])

Q17 = list(df2.loc[0][90:102])

Q18 = list(df2.loc[0][102:109])

Q19 = list(df2.loc[0][109:115])

Q10

In [ ]:
df7 = df[Q7]

df7.columns = ['Python', 'R', 'SQL', 'C', 'C++', 'Java', 'Javascript', 'Julia', 'Swift', 'Bash', 'MATLAB', 'None', 'Other']

plt.figure(figsize=(10,10))
df7.count().sort_values(ascending=True).plot(kind = 'pie');
plt.show();
print(df7.count())

**Python followed by SQL and C++ with 21861, 10757 and 5335 caounts are the top 3 programming languages freuently used**

In [ ]:
#Visualising Q8 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q8',
              data = df,
              order = df['Q8'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q8.value_counts())

**Again Python R, SQL with 20213,1445 and 1338 take the top 3 spot as recommended programming languages to learn for beginners**

In [ ]:
df9 = df[Q9]

df9.columns = ['Jupyter (JupyterLab, Jupyter Notebooks, etc)',' RStudio','Visual Studio','Visual Studio Code (VSCode)',
              'PyCharm','Spyder','Notepad++','Sublime Text','Vim / Emacs','MATLAB','Jupyter Notebook','None','Other']

plt.figure(figsize=(10,10))
df9.count().sort_values(ascending=True).plot(kind = 'pie');
plt.show();
print(df9.count())

**Jupyter studio , Visual Studio and Pycharm with 16234, 10041 and 7469 are the top 3 IDEs used by the respondents**

In [ ]:
df10 = df[Q10]

df10.columns = ['Kaggle Notebooks','Colab Notebooks','Azure Notebooks','Paperspace / Gradient','Binder / JupyterHub','Code Ocean','IBM Watson Studio',
              'Amazon Sagemaker Studio Notebooks','Amazon EMR Notebooks','Google Cloud Notebooks (AI Platform / Vertex AI)','Google Cloud Datalab',
              'Databricks Collaborative Notebooks','Zeppelin / Zepl Notebooks','Deepnote Notebooks','Observable Notebooks','None','Other']

plt.figure(figsize=(20,10))
df10.count().sort_values(ascending=True).plot(kind = 'barh');
plt.show();
print(df10.count())

**Kaggle notebooks, Colab Notebooks followed by None( No notebooks) take top 3 spot with 9508, 9793 and 7175 responses, what's insightful is about 27 % of respondents dont use any notebook , hence a potential target audience for Notebook companies.**

In [ ]:
#Visualising Q11 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q11',
              data = df,
              order = df['Q11'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q11.value_counts())

**Laptops followed by PC/Desktop, Cloud computing platform take top 3 spots with 16231,4916 and 2328 responses, what's insightful is that Deeplearning workstations with 814 have a rising adoption.**

In [ ]:

df12 = df[Q12]

df12.columns = ['NVIDIA GPUs ','Google Cloud TPUs ','AWS Trainium Chips ','AWS Inferentia Chips ','None','Other']

plt.figure(figsize=(20,10))
df12.count().sort_values(ascending=True).plot(kind = 'pie');
plt.show();
print(df12.count())

**Nvidia GPUs followed by Google Cloud TPUs AWS Trianim Chips  tale the top the 3 spots, although None is suppsoed to be at top spot with 13235 responses,Meaning about 50 % respondents dont use any specialised hardware.**

In [ ]:
#Visualising Q13 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q13',
              data = df,
              order = df['Q13'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q13.value_counts())

**Most of the respondents , about 63 % have never used a TPU, another insightful piece since its a unpenetrated audience for TPUs**

In [ ]:
# Visualisisng Q14 now

df14 = df[Q14]

df14.columns = ['Matplotlib ','Seaborn ','Plotly / Plotly Express ','Ggplot / ggplot2 ','Shiny ','D3 js ','Altair ',
               'Bokeh ','Geoplotlib ','Leaflet / Folium ','None','Other']

plt.figure(figsize=(20,10))
df14.count().sort_values(ascending=True).plot(kind = 'pie');
plt.show();
print(df14.count())

**Matplotlib, Seaborn and Plotly / PLotly Express take the top 3 spot with 17596, 12587 and 5779 responses**

In [ ]:
#Visualising Q15 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q15',
              data = df,
              order = df['Q15'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q15.value_counts())

**Evidently ML methods has a huge number of new adopters, with 9163 responses ( about 35 %) have been using it for less than 1 year, furthermore 4675 responses ( about 18 % ) use it for 1 - 2 years. ML methods adotion is on the rise in the last 2 years clearly.**

In [ ]:
# Visualisisng Q16 now

df16 = df[Q16]

df16.columns = ['Scikit-learn ','TensorFlow ','Keras ','PyTorch ','Fast.ai ','MXNet ','Xgboost ','LightGBM ','CatBoost ',
               'Prophet','H2O 3 ','Caret ','Tidymodels ','JAX ','PyTorch Lightning ','Huggingface ','None','Other']

plt.figure(figsize=(20,10))
df16.count().sort_values(ascending=True).plot(kind = 'pie');
plt.show();
print(df16.count())


**Scikit-learn tops the chart with 13988(about 53 %) respondents picking it as a regularly used ML framework, Tesnorflow and Keras with 9372 and 7976 responses tale 2 and 3 spot.**

In [ ]:
# Visualisisng Q17 now

df17 = df[Q17]

df17.columns = ['Linear or Logistic Regression','Decision Trees or Random Forests',
               'Gradient Boosting Machines (xgboost, lightgbm, etc)','Bayesian Approaches','Evolutionary Approaches',
               'Dense Neural Networks (MLPs, etc)','Convolutional Neural Networks','Generative Adversarial Networks',
               'Recurrent Neural Networks','Transformer Networks (BERT, gpt-3, etc)','Selected Choice - None',
               'Selected Choice - Other']

plt.figure(figsize=(20,10))
df17.count().sort_values(ascending=True).plot(kind = 'pie');
plt.show();
print(df17.count())


**When it comes to Algorithm choice Regression(Linear and Logistic) take first spot with 13853 responses( about 53 %) , followed by a close 11864 for Decision Trees or Random Forests ( about 45 %) followed by Gradient boosting ( about 29 %) and a close 27 % for Convolutional Neural Networks** 

In [ ]:
# Visualisisng Q18 now

df18 = df[Q18]

df18.columns =  ['General purpose image/video tools (PIL, cv2, skimage, etc)','Image segmentation methods (U-Net, Mask R-CNN, etc)',
               'Object detection methods (YOLOv3, RetinaNet, etc)',
               'Image classification and other general purpose networks (VGG, Inception, ResNet, ResNeXt, NASNet, EfficientNet, etc)',
               'Generative Networks (GAN, VAE, etc)','None','Other']

plt.figure(figsize=(10,10))
df18.count().sort_values(ascending=True).plot(kind = 'barh');
plt.show();
print(df18.count())

**When it comes to computer vision Image classification and general purpose networks with 4374 take the top spot, foloowed by Image segmentation methods and object detetction methods with 4374 and 2741** 

In [ ]:
# Visualisisng Q19 now

df19 = df[Q19]

df19.columns = [' Word embeddings/vectors (GLoVe, fastText, word2vec)','Encoder-decorder models (seq2seq, vanilla transformers)',
               'Contextualized embeddings (ELMo, CoVe)','Transformer language models (GPT-3, BERT, XLnet, etc)',
               'None','Other']

plt.figure(figsize=(20,10))
df19.count().sort_values(ascending=True).plot(kind = 'barh');
plt.show();
print(df19.count())



**When it comes to Choice of NLP methods, word embeddings with 2644 takes top spot follwed by Transformer language models with 2352 and Encoder-decoder models with 2024 responses**

In [ ]:
#Visualising Q20 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q20',
              data = df,
              order = df['Q20'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q20.value_counts())

**Computer/Tech with 4079 followed by Academics/Education - 3214 and Accounting/Finance with 1459 take the top 3 spots interms of Current/last industry respondents belong to**

In [ ]:
#Visualising Q21 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q21',
              data = df,
              order = df['Q21'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q21.value_counts())

In [ ]:
#Visualising Q22 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q22',
              data = df,
              order = df['Q22'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q22.value_counts())

**This is interesting to note that 1 - 2 employees share Data sceince workload in 3642 cases from our respondents, ironically second spot goes to organisations with 20 + employees in the organisation, this shows disparity in Data science strategy adopted by various organisation, while the ones with small number of emplyees could be experimenting to get hands on experience , teams with 20 + people in all probablity are more mature in terms of adoption of Data Science.**

In [ ]:
# Now we get down to consolidating data for rest of the questions with multiple parts to it

Q24 = list(df2.loc[0][119:127])

Q27A = list(df2.loc[0][129:141])

Q29 = list(df2.loc[0][142:147])

Q30 = list(df2.loc[0][147:155])

Q31 = list(df2.loc[0][155:165])

Q32 = list(df2.loc[0][165:186])

Q34 = list(df2.loc[0][187:204])

Q36 = list(df2.loc[0][205:213])

Q37 = list(df2.loc[0][213:221])

Q38 = list(df2.loc[0][221:233])

Q39 = list(df2.loc[0][233:243])

Q40 = list(df2.loc[0][243:255])

Q42 = list(df2.loc[0][256:268])

Q27B = list(df2.loc[0][268:280])

Q29B = list(df2.loc[0][280:285])

Q30B = list(df2.loc[0][285:293])

Q31B = list(df2.loc[0][293:303])

Q32B = list(df2.loc[0][303:324])

Q34B = list(df2.loc[0][324:341])

Q36B = list(df2.loc[0][341:349])

Q37B = list(df2.loc[0][349:357])

Q38B = list(df2.loc[0][357:369])
#for example lets see list of all parts for Q38
Q38B 

In [ ]:
df24 = df[Q24]
df24.columns = ['Analyze and understand data to influence product or business decisions',
               'Build and/or run the data infrastructure that my business uses for storing, analyzing, and operationalizing data',
               'Build prototypes to explore applying machine learning to new areas',
               'Build and/or run a machine learning service that operationally improves my product or workflows',
               'Experimentation and iteration to improve existing ML models',
               'Do research that advances the state of the art of machine learning',
               'None of these activities are an important part of my role at work','Other']

plt.figure(figsize=(20,10))
df24.count().sort_values(ascending=True).plot(kind = 'barh');
plt.show();
print(df24.count())

In [ ]:
#Visualising Q25 now 
sn.set(style='darkgrid')
plt.figure(figsize=(10,10))
sn.countplot(y = 'Q25',
              data = df,
              order = df['Q25'].value_counts().index)
plt.xticks(size = 12)
plt.show()

print(df.Q25.value_counts())

In [ ]:
print(df_In.Q25.value_counts(),"\n")

print(df_US.Q25.value_counts(),"\n")

print(df_Cn.Q25.value_counts(),"\n")

print(df_Jp.Q25.value_counts(),"\n")

In [ ]:
#Side by side plotting of India,US and Japan Salary distribution
sn.set(style='darkgrid')

fig, axs = plt.subplots(ncols=3,figsize=(35,30))

sn.countplot(y = 'Q25',
              data = df_In,ax=axs[0],
              order = df_In['Q25'].value_counts().index)
sn.countplot(y = 'Q25',
              data = df_US,ax=axs[1],
              order = df_US['Q25'].value_counts().index)
sn.countplot(y = 'Q25',
              data = df_Jp,ax=axs[2],
              order = df_Jp['Q25'].value_counts().index)

## This is work in progress, will have more updates continuing from here...